In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('s2-in.txt', sep='\t')
df.head()
text_combined = df['Text'].str.cat(sep=' ')

In [3]:
# build dictionary v. 1.0, maybe unnecessary? 完全不必要，用Keras的Tokenizer就行了
vocab = {}
i = 1

for word in text_combined.split():
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1

In [4]:
# separator v. 1.0
def tokenize(text):
    return text.split()
tokens = tokenize(text_combined)
len(tokens)
tokens[:10]

['WMLK>',
 'DWJD',
 'S>B',
 'W<L',
 'BC"NJ>',
 'WMKSJN',
 'HWW',
 'LH',
 'BLBWC">',
 'WL>']

In [5]:
# organize into sequences of tokens
train_len = 25+1 # 25 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [6]:
# Tokenize the sequences

from keras.preprocessing.text import Tokenizer

# integer encode sequences of words
tokenizer = Tokenizer(lower=False) #ETCBC data is in uppercase, we need to keep it that way
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
print(tokenizer.index_word[:10]) #index_word is a dictionary where keys are indices and values are words
vocabulary_size = len(tokenizer.word_index) #总共有多少词形
vocabulary_size

TypeError: unhashable type: 'slice'

In [6]:
import numpy as np
sequences = np.array(sequences) #converts the Python list into a NumPy array.

In [7]:
#formalize the data and labels. The data will be all the words in the sequence except the last one, and the label will be the last word in the sequence.

import tensorflow as tf
from keras.utils import to_categorical

X = sequences[:,:-1] #all rows, all columns except the last one
y = sequences[:,-1] #all rows, last column only
y_one_hot = to_categorical(y, num_classes=vocabulary_size+1) #one-hot encoding
seq_len = X.shape[1]
y_one_hot.shape

(77974, 18035)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
def creat_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

model = creat_model(vocabulary_size+1, seq_len)

2024-03-30 23:58:08.668734: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-03-30 23:58:08.810199: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-30 23:58:08.810927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-30 23:58:08.8

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            450875    
                                                                 
 lstm (LSTM)                 (None, 25, 50)            15200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 18035)             919785    
                                                                 
Total params: 1,408,610
Trainable params: 1,408,610
Non-trainable params: 0
_________________________________________________________________


2024-03-30 23:58:08.893274: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-30 23:58:08.894108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-30 23:58:08.894661: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [9]:
from pickle import dump, load
model.fit(X, y_one_hot, batch_size=128, epochs=500, verbose=1)

Epoch 1/500


2024-03-30 23:58:11.132511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-30 23:58:11.133524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-30 23:58:11.134215: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

610/610 [==============================] - 17s 26ms/step - loss: 8.6057 - accuracy: 0.0213
Epoch 2/500
610/610 [==============================] - 16s 26ms/step - loss: 8.1861 - accuracy: 0.0220
Epoch 3/500
610/610 [==============================] - 16s 26ms/step - loss: 8.0568 - accuracy: 0.0233
Epoch 4/500
610/610 [==============================] - 16s 26ms/step - loss: 7.8946 - accuracy: 0.0241
Epoch 5/500
610/610 [==============================] - 16s 26ms/step - loss: 7.7003 - accuracy: 0.0264
Epoch 6/500
610/610 [==============================] - 15s 25ms/step - loss: 7.5109 - accuracy: 0.0280
Epoch 7/500
610/610 [==============================] - 16s 26ms/step - loss: 7.3404 - accuracy: 0.0317
Epoch 8/500
610/610 [==============================] - 16s 26ms/step - loss: 7.1697 - accuracy: 0.0351
Epoch 9/500
610/610 [==============================] - 16s 26ms/step - loss: 6.9982 - accuracy: 0.0402
Epoch 10/500
610/610 [==============================] - 16s 25ms/step - loss: 6.8251 

In [1]:
model.save('s2-out.h5')
dump(tokenizer, open('s2-out', 'wb'))

NameError: name 'model' is not defined

In [6]:
from keras.models import load_model
from pickle import load
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict class probabilities for each word
        pred_probabilities = model.predict(pad_encoded, verbose=0)[0]

        # Get the index of the word with the highest probability
        pred_word_ind = np.argmax(pred_probabilities)
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [12]:

seed_text = "DNTL LK >R<> DMRDJ> XLB> WDBC> CM< >JSRJL MRJ> >LHN MRJ> XD HW RXM LMRJ> >LHK MN KLH LBK WMN KLH^ NPCK WMN KLH QNJNK"

len(seed_text.split())
model = load_model('s2-out.h5')
tokenizer = load(open('s2-out', 'rb'))

generate_text(model,tokenizer,seq_len=25,seed_text=seed_text,num_gen_words=25)

2024-03-31 08:28:00.416074: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-31 08:28:00.416683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-31 08:28:00.417400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

'LJV MN DKTJB WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ> WLTWNJ>'